In [ ]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nrrd

output_size =[128, 128, 64]

def covert_h5(glob_str, old_replaced, new_replaced):
    """
    备注：不要骨头，骨头合并到背景类别中
    """
    listt = glob(glob_str)
    #listt = glob('../../data/2018LA_Seg_Training Set/*/CTM.nrrd')
    #listt = glob('../../data/2018LA_Seg_Training Set/*/lgemri.nrrd')
    for item in listt:#tqdm(listt):
        print(item.split('/')[-2],':')#win系统改为'\\'
        
        image, img_header = nrrd.read(item)
        label, label_header = nrrd.read(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        seg, seg_header = nrrd.read(item.replace(old_replaced, 'Segmentation.seg.nrrd'))
        
        offset=[]
        for k in seg_header['Segmentation_ReferenceImageExtentOffset'].split():
            offset += [int(k)]
        sizes = seg_header['sizes'][1::]
        
        image = image[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]]
        label = label[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]].astype(np.uint8)
        
        # 类别名称和顺序
        target_name = ['dura','bone','SC']#目标类别顺序
        label_name = [
            seg_header['Segment0_Name'],
            seg_header['Segment1_Name'],
            seg_header['Segment2_Name'] 
            ]#人工标注的类别顺序
        ## 调整顺序，注意：seg是onehot编码
        idx = [label_name.index(name) for name in target_name]
        seg = seg[idx]
        ## 补充背景类别
        bg = np.ones(seg.shape[1:], dtype=np.uint8)-seg.sum(axis=0, dtype=np.uint8)
        bg = bg[np.newaxis,:]
        seg = np.concatenate((bg,seg),axis=0)
        assert np.unique(bg) == [0, 1], "pixel class error"
        ## 转化为非onehot编码
        label = np.argmax(seg, axis=0)
        
        # 合并骨头到背景中
        label[label==2] = 0
        label[label==3] = 2
        
        # 缩小图像
#         image = image[0:-1:2,0:-1:2,:]
#         label = label[0:-1:2,0:-1:2,:]
        
        # cut
        tempL = np.nonzero(label)
        minx, maxx = np.min(tempL[0]), np.max(tempL[0])
        miny, maxy = np.min(tempL[1]), np.max(tempL[1])
        minz, maxz = np.min(tempL[2]), np.max(tempL[2])
        
        w, h, d = image.shape
        print(minx, maxx, miny, maxy, minz, maxz)
        print(image.shape)
        print((maxx - minx)/w, (maxy - miny)/h, (maxz - minz)/d)
        
#         w, h, d = label.shape
#         px = max(output_size[0] - (maxx - minx), 0) // 2
#         py = max(output_size[1] - (maxy - miny), 0) // 2
#         pz = max(output_size[2] - (maxz - minz), 0) // 2
#         minx = max(minx - np.random.randint(10, 20) - px, 0)
#         maxx = min(maxx + np.random.randint(10, 20) + px, w)
#         miny = max(miny - np.random.randint(10, 20) - py, 0)
#         maxy = min(maxy + np.random.randint(10, 20) + py, h)
#         minz = max(minz - np.random.randint(5, 10) - pz, 0)
#         maxz = min(maxz + np.random.randint(5, 10) + pz, d)
#         image = (image - np.mean(image)) / np.std(image)
#         image = image.astype(np.float32)
#         image = image[minx:maxx, miny:maxy]
#         label = label[minx:maxx, miny:maxy]
#         print(label.shape)
        
#         f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
#         f.create_dataset('image', data=image, compression="gzip")
#         f.create_dataset('label', data=label, compression="gzip")
#         f.close()
    print("total number of seg-samples:", len(listt))

def covert_h5_unseg(glob_str, old_replaced, new_replaced):
    """
    备注：无标注数据的格式转换
    """
    listt = glob(glob_str)
#     import pdb
#     pdb.set_trace()
    for item in tqdm(listt):
        print(item.split('/')[-2],':')
        # 读取原始图
        image, img_header = nrrd.read(item)     
#         # 缩小图像
#         image = image[0:-1:2,0:-1:2,:]
        # 标准化
        image = (image - np.mean(image)) / np.std(image)
        image = image.astype(np.float32)
        print(image.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.close()  
    print("total number of unseg-samples:", len(listt))

    
if __name__ == '__main__':
    # 有标签数据
    print('seg dataset:')
    glob_str = '../../data/CTM_dataset/Segmented/*/CTM.nrrd'
    covert_h5(glob_str,'CTM.nrrd','mri_norm2_core.h5')
    # 无标签数据
    print('unseg dataset:')
    glob_str = '../../data/CTM_dataset/unSegmented/*/CTM.nrrd'
    covert_h5_unseg(glob_str,'CTM.nrrd','mri_norm2.h5')
    glob_str = '../../data/CTM_dataset/unSegmented/*/CT-vol.nrrd'
    covert_h5_unseg(glob_str,'CT-vol.nrrd','mri_norm2_core.h5')      


seg dataset:
845929 :
227 292 53 155 15 91
(512, 381, 92)
0.126953125 0.2677165354330709 0.8260869565217391
1559027 :
203 293 229 392 4 72
(512, 480, 73)
0.17578125 0.33958333333333335 0.9315068493150684
1433611 :
227 307 162 233 9 64
(512, 291, 65)
0.15625 0.24398625429553264 0.8461538461538461
B1027338 :
208 309 95 229 8 84
(512, 496, 85)
0.197265625 0.2701612903225806 0.8941176470588236
936932-dingzi :
217 279 131 241 11 84
(512, 336, 85)
0.12109375 0.3273809523809524 0.8588235294117647
1371504 :
213 309 257 318 35 84
(512, 464, 85)
0.1875 0.1314655172413793 0.5764705882352941
785777 :
165 291 228 370 14 84
(512, 477, 85)
0.24609375 0.2976939203354298 0.8235294117647058
1632110 :
197 289 232 364 6 78
(510, 489, 79)
0.1803921568627451 0.26993865030674846 0.9113924050632911
1673882 :
197 289 234 358 4 84
(512, 409, 85)
0.1796875 0.30317848410757947 0.9411764705882353
B759663 :
150 298 96 297 1 84
(512, 346, 85)
0.2890625 0.5809248554913294 0.9764705882352941
B1673882 :
197 289 234 358

230 303 55 179 8 70
(512, 352, 71)
0.142578125 0.3522727272727273 0.8732394366197183
B1709234 :
231 296 183 259 5 70
(512, 395, 71)
0.126953125 0.19240506329113924 0.9154929577464789
1330032 :
217 292 185 266 10 53
(512, 354, 54)
0.146484375 0.2288135593220339 0.7962962962962963
B1181908 :
214 292 242 336 5 74
(512, 394, 75)
0.15234375 0.23857868020304568 0.92
B772230 :
226 309 219 342 8 43
(512, 448, 44)
0.162109375 0.27455357142857145 0.7954545454545454
1613978 :
228 275 210 277 26 64
(509, 428, 65)
0.09233791748526522 0.15654205607476634 0.5846153846153846
685245 :
233 291 253 344 7 73
(512, 395, 74)
0.11328125 0.23037974683544304 0.8918918918918919
B1650523 :
215 288 243 331 15 84
(512, 418, 85)
0.142578125 0.21052631578947367 0.8117647058823529
B872077 :
208 290 138 256 7 51
(512, 336, 52)
0.16015625 0.35119047619047616 0.8461538461538461
614745 :
235 310 242 315 10 70
(512, 437, 71)
0.146484375 0.16704805491990846 0.8450704225352113
1181908 :
214 292 242 336 5 74
(512, 394, 75)
0

227 331 236 355 2 73
(512, 447, 74)
0.203125 0.2662192393736018 0.9594594594594594
b1700637-neck :
209 298 187 332 0 71
(510, 512, 37)
0.17450980392156862 0.283203125 1.9189189189189189
b1371504 :
213 310 257 318 35 84
(512, 464, 85)
0.189453125 0.1314655172413793 0.5764705882352941
1536553 :
198 294 262 360 6 84
(512, 443, 85)
0.1875 0.22121896162528218 0.9176470588235294
B845929 :
227 292 53 155 15 91
(512, 381, 92)
0.126953125 0.2677165354330709 0.8260869565217391
B1425167 :
244 301 230 298 8 84
(511, 419, 85)
0.11154598825831702 0.162291169451074 0.8941176470588236
1713865 :
208 289 219 339 3 76
(512, 413, 77)
0.158203125 0.29055690072639223 0.948051948051948
B820056 :
200 299 234 321 8 84
(509, 480, 85)
0.1944990176817289 0.18125 0.8941176470588236
B809338 :
238 365 264 366 12 84
(512, 463, 85)
0.248046875 0.2203023758099352 0.8470588235294118
930124 :
195 292 151 271 9 78
(512, 371, 79)
0.189453125 0.32345013477088946 0.8734177215189873
1298116 :
202 344 242 429 12 77
(512, 507, 

# 数据集划分&生成*.list文件

In [ ]:
# from dataset_split import dataset_split, make_dataset_list
# # 有标签数据
# save_dir = '../../data/CTM_dataset'
# dataset_dir = '../../data/CTM_dataset/Segmented'
# dataset_split(path=dataset_dir,save_dir=save_dir)
# # 无标签数据
# dataset_dir = '../../data/CTM_dataset/unSegmented'
# make_dataset_list(path=dataset_dir,save_dir=save_dir)